In [ ]:
! pip install datasets evaluate rouge-score nltk

Then you need to install Git-LFS. Uncomment the following instructions:

In [ ]:
!apt install git-lfs

Make sure your version of Transformers is at least 4.11.0 since the functionality was introduced in that version:

In [ ]:
import transformers

print(transformers.__version__)

4.30.0


In [ ]:
model_checkpoint = "t5-small"

## Loading the dataset

 [🤗 Datasets](https://github.com/huggingface/datasets) library is used to download the data and get the metric we need to use for evaluation (to compare our model to the benchmark). This can be easily done with the functions `load_dataset` and `load_metric`.  

In [ ]:
import pandas as pd
from google.colab import drive
from datasets import load_dataset
from evaluate import load

#raw_datasets = load_dataset("xsum")
metric = load("rouge")

The `dataset` object itself is [`DatasetDict`](https://huggingface.co/docs/datasets/package_reference/main_classes.html#datasetdict), which contains one key for the training, validation and test set:

In [ ]:
# Mount Google Drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import datasets
from datasets import DatasetDict

In [ ]:
dataset = load_dataset('csv',data_files= "/content/drive/MyDrive/fyp/dataset/ComplaintAndSummaryoldnew (2).csv")

total_rows = 600  # Total number of rows in the dataset

# Calculate the number of rows for each split
num_train = int(total_rows * 0.70)  # 70% for training
num_eval = int(total_rows * 0.20)   # 20% for evaluation
num_test = total_rows - num_train - num_eval  # Remaining 10% for testing

# Select rows for each split
train_dataset = dataset['train'].select(range(num_train))  # Select 70% of the data for training
eval_dataset = dataset['train'].select(range(num_train, num_train + num_eval))  # Select 20% of the data for evaluation
test_dataset = dataset['train'].select(range(num_train + num_eval, num_train + num_eval + num_test))  # Select 10% of the data for testing



In [ ]:
# Create a DatasetDict
dataset_dict = DatasetDict({
    'train': train_dataset,
    'eval': eval_dataset,
    'test': test_dataset
})

In [ ]:
dataset_dict

To access an actual element, you need to select a split first, then give an index:

In [ ]:
dataset_dict["train"][0]

To get a sense of what the data looks like, the following function will show some examples picked randomly in the dataset.

In [ ]:
import datasets
import random
import pandas as pd
from IPython.display import display, HTML

def show_random_elements(dataset, num_examples=5):
    assert num_examples <= len(dataset), "Can't pick more elements than there are in the dataset."
    picks = []
    for _ in range(num_examples):
        pick = random.randint(0, len(dataset)-1)
        while pick in picks:
            pick = random.randint(0, len(dataset)-1)
        picks.append(pick)

    df = pd.DataFrame(dataset[picks])
    for column, typ in dataset.features.items():
        if isinstance(typ, datasets.ClassLabel):
            df[column] = df[column].transform(lambda i: typ.names[i])
    display(HTML(df.to_html()))

In [ ]:
show_random_elements(dataset_dict["train"])

,complaint,summary
0,"THE CONTACT STATED WHILE DRIVING THE VEHICLE SHUT OFF WITHOUT WARNING, AND IT WOULD NOT RESTART.",Vehicle shut off while driving without warning and would not restart.
1,"Engine surge while coming to a stop, leading to collision with interstate sign, PCM replacement recommended by dealer.","Engine surge during braking, resulting in collision, PCM replacement suggested by dealer."
2,"Body mounts on left front loosen repeatedly, tightened by dealer multiple times, manufacturer buying back vehicle.","Repeated loosening of left front body mounts, unresolved by dealer repairs, manufacturer repurchasing vehicle."
3,"Steering becomes impossible to turn in any direction, known problem with defective seal reported by dealer.","Steering becomes impossible due to defective seal, dealer diagnosis."
4,"Vehicle rolling backwards unexpectedly after engine start while gear shift indicator in park, dealership not examined for problem.","Vehicle rolling backwards unexpectedly after engine start, dealership not yet examined for issue."


In [ ]:
show_random_elements(dataset_dict["eval"])

In [ ]:
show_random_elements(dataset_dict["test"])

The metric is an instance of [`datasets.Metric`](https://huggingface.co/docs/datasets/package_reference/main_classes.html#datasets.Metric):

In [ ]:
metric

## Preprocessing the data

Before we can feed those texts to our model, we need to preprocess them. This is done by a 🤗 Transformers `Tokenizer` which will (as the name indicates) tokenize the inputs (including converting the tokens to their corresponding IDs in the pretrained vocabulary) and put it in a format the model expects, as well as generate the other inputs that the model requires.

To do all of this, we instantiate our tokenizer with the `AutoTokenizer.from_pretrained` method, which will ensure:

- we get a tokenizer that corresponds to the model architecture we want to use,
- we download the vocabulary used when pretraining this specific checkpoint.

That vocabulary will be cached, so it's not downloaded again the next time we run the cell.

In [ ]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)

You can directly call this tokenizer on one sentence or a pair of sentences:

In [ ]:
tokenizer("Hello, this one sentence!")

{'input_ids': [8774, 6, 48, 80, 7142, 55, 1], 'attention_mask': [1, 1, 1, 1, 1, 1, 1]}

If you are using one of the five T5 checkpoints we have to prefix the inputs with "summarize:" (the model can also translate and it needs the prefix to know which task it has to perform).

In [ ]:
if model_checkpoint in ["t5-small", "t5-base", "t5-larg", "t5-3b", "t5-11b"]:
    prefix = "summarize: "
else:
    prefix = ""

We can then write the function that will preprocess our samples. We just feed them to the `tokenizer` with the argument `truncation=True`. This will ensure that an input longer that what the model selected can handle will be truncated to the maximum length accepted by the model. The padding will be dealt with later on (in a data collator) so we pad examples to the longest length in the batch and not the whole dataset.

In [ ]:
max_input_length = 1024
max_target_length = 128

def preprocess_function(examples):
    inputs = [prefix + doc for doc in examples['complaint']]
    model_inputs = tokenizer(inputs, max_length=max_input_length, truncation=True)

    # Setup the tokenizer for targets
    labels = tokenizer(text_target=examples[' summary'], max_length=max_target_length, padding=True,truncation=True)

    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

This function works with one or several examples. In the case of several examples, the tokenizer will return a list of lists for each key:

In [ ]:
preprocess_function(dataset_dict['train'][:2])

To apply this function on all the pairs of sentences in our dataset, we just use the `map` method of our `dataset` object we created earlier. This will apply the function on all the elements of all the splits in `dataset`, so our training, validation and testing data will be preprocessed in one single command.

In [ ]:
tokenized_datasets = dataset_dict.map(preprocess_function, batched=True)

Even better, the results are automatically cached by the 🤗 Datasets library to avoid spending time on this step the next time you run your notebook. The 🤗 Datasets library is normally smart enough to detect when the function you pass to map has changed (and thus requires to not use the cache data). For instance, it will properly detect if you change the task in the first cell and rerun the notebook. 🤗 Datasets warns you when it uses cached files, you can pass `load_from_cache_file=False` in the call to `map` to not use the cached files and force the preprocessing to be applied again.

Note that we passed `batched=True` to encode the texts by batches together. This is to leverage the full benefit of the fast tokenizer we loaded earlier, which will use multi-threading to treat the texts in a batch concurrently.

## Fine-tuning the model

Now that the data is ready, we can download the pretrained model and fine-tune it. Since our task is of the sequence-to-sequence kind, we use the `AutoModelForSeq2SeqLM` class. Like with the tokenizer, the `from_pretrained` method will download and cache the model for us.

In [ ]:
from transformers import AutoModelForSeq2SeqLM, DataCollatorForSeq2Seq, Seq2SeqTrainingArguments, Seq2SeqTrainer

model = AutoModelForSeq2SeqLM.from_pretrained(model_checkpoint)

Note that  we don't get a warning like in our classification example. This means we used all the weights of the pretrained model and there is no randomly initialized head in this case.

To instantiate a `Seq2SeqTrainer`, we will need to define three more things. The most important is the [`Seq2SeqTrainingArguments`](https://huggingface.co/transformers/main_classes/trainer.html#transformers.Seq2SeqTrainingArguments), which is a class that contains all the attributes to customize the training. It requires one folder name, which will be used to save the checkpoints of the model, and all other arguments are optional:

In [ ]:
! pip install -U accelerate
! pip install transformers==4.30

In [ ]:
import accelerate
import transformers

transformers.__version__, accelerate.__version__

('4.30.0', '0.29.2')

In [ ]:
batch_size = 20
model_name = model_checkpoint.split("/")[-1]
args = Seq2SeqTrainingArguments(
   f"{model_name}-finetuned-vehidefecttwo",
   evaluation_strategy = "epoch",
   learning_rate=2e-5,
   per_device_train_batch_size=batch_size,
   per_device_eval_batch_size=batch_size,
   weight_decay=0.01,
   save_total_limit=3,
   num_train_epochs=8,
   predict_with_generate=True,
   fp16=True,
   push_to_hub=True,
)

Here we set the evaluation to be done at the end of each epoch, tweak the learning rate, use the `batch_size` defined at the top of the cell and customize the weight decay. Since the `Seq2SeqTrainer` will save the model regularly and our dataset is quite large, we tell it to make three saves maximum. Lastly, we use the `predict_with_generate` option (to properly generate summaries) and activate mixed precision training (to go a bit faster).



In [ ]:
data_collator = DataCollatorForSeq2Seq(tokenizer, model=model)

The last thing to define for our `Seq2SeqTrainer` is how to compute the metrics from the predictions. We need to define a function for this, which will just use the `metric` we loaded earlier, and we have to do a bit of pre-processing to decode the predictions into texts:

In [ ]:
import nltk
import numpy as np

def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    decoded_preds = tokenizer.batch_decode(predictions, skip_special_tokens=True)
    # Replace -100 in the labels as we can't decode them.
    labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)

    # Rouge expects a newline after each sentence
    decoded_preds = ["\n".join(nltk.sent_tokenize(pred.strip())) for pred in decoded_preds]
    decoded_labels = ["\n".join(nltk.sent_tokenize(label.strip())) for label in decoded_labels]

    # Note that other metrics may not have a `use_aggregator` parameter
    # and thus will return a list, computing a metric for each sentence.
    result = metric.compute(predictions=decoded_preds, references=decoded_labels, use_stemmer=True, use_aggregator=True)
    # Extract a few results
    result = {key: value * 100 for key, value in result.items()}

    # Add mean generated length
    prediction_lens = [np.count_nonzero(pred != tokenizer.pad_token_id) for pred in predictions]
    result["gen_len"] = np.mean(prediction_lens)

    return {k: round(v, 4) for k, v in result.items()}

Then we just need to pass all of this along with our datasets to the `Seq2SeqTrainer`:

In [ ]:
!echo {tokenized_datasets.keys()}

In [ ]:
eval_dataset = tokenized_datasets.get("eval")

In [ ]:
print(eval_dataset)

Dataset({
    features: ['complaint', ' summary', 'input_ids', 'attention_mask', 'labels'],
    num_rows: 120
})


In [ ]:
train_dataset = tokenized_datasets.get("train")

In [ ]:
print(train_dataset)

Dataset({
    features: ['complaint', ' summary', 'input_ids', 'attention_mask', 'labels'],
    num_rows: 420
})


In [ ]:
from huggingface_hub import notebook_login

notebook_login()

In [ ]:
trainer = Seq2SeqTrainer(
    model,
    args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["eval"],
    data_collator=data_collator,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)

We can now finetune our model by just calling the `train` method:

In [ ]:
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [ ]:
trainer.train()

/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:411: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
You're using a T5TokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Epoch,Training Loss,Validation Loss,Rouge1,Rouge2,Rougel,Rougelsum,Gen Len
1,No log,7.207804,52.564000,34.178400,48.189200,48.227800,15.650000
2,No log,3.753271,49.843300,32.315300,45.878000,45.791500,15.258300
3,No log,1.794542,19.211300,11.753700,17.797800,17.473600,7.425000
4,No log,1.349722,7.345400,4.354000,6.840000,6.723300,3.508300
5,No log,1.244936,2.126200,0.850500,1.891500,1.871600,1.575000
6,No log,1.235466,0.534200,0.129000,0.461500,0.444400,0.633300
7,No log,1.235229,0.378000,0.069400,0.313900,0.313900,0.475000
8,No log,1.232922,0.320500,0.069400,0.256400,0.256400,0.316700


TrainOutput(global_step=168, training_loss=3.7580642700195312, metrics={'train_runtime': 53.075, 'train_samples_per_second': 63.307, 'train_steps_per_second': 3.165, 'total_flos': 222050429829120.0, 'train_loss': 3.7580642700195312, 'epoch': 8.0})

You can now upload the result of the training to the Hub, just execute this instruction:

In [ ]:
trainer.push_to_hub()

In [ ]:
from huggingface_hub import notebook_login

notebook_login()

You can now share this model with all your friends, family, favorite pets: they can all load it with the identifier `"your-username/the-name-you-picked"` so for instance:

```python
from transformers import AutoModelForSeq2SeqLM

model = AutoModelForSeq2SeqLM.from_pretrained("sgugger/my-awesome-model")
```

In [ ]:
model_name = "ItsMayur/t5-small-finetuned-vehidefecttwo"  # e.g., "t5-small-finetuned-vehidefect"
model = AutoModelForSeq2SeqLM.from_pretrained(model_name)
tokenizer = AutoTokenizer.from_pretrained(model_name)

config.json:   0%|          | 0.00/1.47k [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/242M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/2.32k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.42M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/2.20k [00:00<?, ?B/s]

In [ ]:
max_input_length = 1000
max_target_length = 128
input_text = "Engine Bypass Coolant Valve. Issue could cause coolant leak and cause car to overheat. Issue confirmed by Toyota dealership. Also many individuals confirming the same issue. Component inspected by dealership. No issues until component failed and now a constant engine repair message."

In [ ]:
# Encoding
encoding = tokenizer(input_text, truncation=True, padding="longest", max_length=max_input_length, return_tensors="pt")
input_ids = encoding["input_ids"].squeeze()

In [ ]:
encoding = tokenizer(input_text, truncation=True, padding="longest", max_length=max_input_length, return_tensors="pt")
input_ids = encoding["input_ids"][0] if isinstance(encoding["input_ids"], tuple) else encoding["input_ids"]

In [ ]:
# Model forward pass & Generation
outputs = model.generate(input_ids, max_length=max_target_length, num_beams=4, early_stopping=True, pad_token_id=tokenizer.pad_token_id)
generated_summary = tokenizer.decode(outputs[0], skip_special_tokens=True)

In [ ]:
print(f"Generated Summary:\n{generated_summary}")